# Lidar to Camera Projection

Before starting this tutorial, you'll need to have at least one sequence from the Boreas dataset downloaded.
If you're working on a local machine, follow these steps to download a sequence:
1. [Create an AWS account](https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/)
2. [Install the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html)
3. Create a `root` folder to store the dataset, example: `/path/to/data/boreas/` Each sequence will then be a folder under `root`.
4. Use the AWS CLI to download a sequence:
```
root=/path/to/data/boreas/
sequence=boreas-2021-09-02-11-42
aws s3 sync s3://boreas/$sequence $root$sequence
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyboreas import BoreasDataset
from pyboreas.utils.utils import get_inverse_tf

root = '/path/to/data/boreas/'
split = None
# AWS: Note: Free Tier SageMaker instances don't have enough storage (25 GB) for 1 sequence (100 GB)
# root = '/home/ec2-user/SageMaker/boreas/'
# split = [['boreas-2021-09-02-11-42', 163059759e6, 163059760e6-1]]

bd = BoreasDataset(root, split=split, verbose=True)

## Removing Motion Distortion
`body_rate` is the velocity of the sensor in the sensor frame, $\boldsymbol{\varpi}$

$\Delta t = t - t_{\text{ref}}$ is the time difference between a given measurement time `t` and a reference time, `tref`.

$$
\mathbf{T}_{\text{undistort}} = \exp(\Delta t \boldsymbol{\varpi}^\wedge)
$$

Convenience function: `PointCloud.remove_motion()`

## Transform from Lidar to Camera
$$
\mathbf{x}_{c} = \mathbf{T}_{\text{enu_camera}}^{-1} \mathbf{T}_{\text{enu_lidar}} \mathbf{x}_{l}
$$

Convenience function: `PointCloud.transform()`

## Project onto Image Plane

$$
\begin{bmatrix} u \\ v \end{bmatrix} = \mathbf{D}~\mathbf{P}~\frac{1}{z}~\mathbf{x}_c
$$

$$
\text{where} ~~ \mathbf{D} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \end{bmatrix}, ~ \mathbf{P} = \begin{bmatrix} f_u & 0 & c_u & 0 \\ 0 & f_v & c_v & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
$$

Convenience function: `PointCloud.project_onto_image()`

In [ ]:
%matplotlib inline
import time
seq = bd.sequences[0]
seq.synchronize_frames(ref='camera')  # simulates having synchronous measurements
idx = 0  # try different frame indices!
cam = seq.get_camera(idx)
lid = seq.get_lidar(idx)

# Remove motion distortion from pointcloud:
print('body rate in lidar frame:')
print(lid.body_rate)
lid.remove_motion(lid.body_rate)

# Get the transform from lidar to camera:
T_enu_camera = cam.pose
T_enu_lidar = lid.pose
T_camera_lidar = np.matmul(get_inverse_tf(T_enu_camera), T_enu_lidar)
print('T_camera_lidar:')
print(T_camera_lidar)
lid.transform(T_camera_lidar)

# Remove points outside our region of interest
lid.passthrough([-75, 75, -20, 10, 0, 40])  # xmin, xmax, ymin, ymax, zmin, zmax

# Project lidar points onto the camera image, using the projection matrix, P0.
uv, colors = lid.project_onto_image(seq.calib.P0)

# Draw the projection
fig = plt.figure(figsize=(24.48, 20.48), dpi=100)
ax = fig.add_subplot()
ax.imshow(cam.img[:, :, ::-1])
ax.set_xlim(0, 2448)
ax.set_ylim(2048, 0)
ax.scatter(uv[:, 0], uv[:, 1], c=colors, marker=',', s=3, edgecolors='none', alpha=0.7, cmap='jet')
ax.set_axis_off()
plt.show()